In [5]:
Company.objects.all() 

<QuerySet [<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>]>

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [6]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [7]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [8]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [9]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [10]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [11]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [12]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [13]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
    if isinstance(road,str):
        roads = Road.objects.filter(company=company,name__icontains=road).annotate(distance=Distance(point, 'path')).order_by('distance')
        road = roads[0]
    else:
        road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [14]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [15]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [16]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [17]:
def get_sign_color(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [18]:
def get_sign_film(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.upper().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [19]:
!cat .env

STAGE=CCR_PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [20]:
filename='Planilha de Inventário Drenagem Profunda OAC - RioSP_RevFinal - 01.11.20... (1) (1)'
# road_name='470'

Escolhendo a aba da planilha do Excel

In [22]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

Testando planilha carregada do Excel

In [23]:
print(ws['A1'].value)

Identificação ANTT


Carregando os valores no script

In [24]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

Definindo valores da importação

In [45]:
company = Company.objects.get(name='CCR - Rio Sp')

#Classe Bueiros
occurrence_type_ficha = OccurrenceType.objects.get(name='Drenagem Profunda Ficha ANTT',company=company)
occurrence_type_elemento = OccurrenceType.objects.get(name='Drenagem Profunda Elemento',company=company)

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
firm=Firm.objects.get(name='Civil: SGOAC',company=company)
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
company

<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>

In [26]:
# road=Road.objects.get(company=company,name__icontains="470")
# items=list(road.marks.items())
# first_km=items[0][1]['km']
# last_km=items[-1][1]['km']
# print(first_km,last_km)

In [27]:
len(values),values[0]

(8036,
 {'Identificação ANTT': 'BU 116 RJ 215+140 N 1',
  'km': '215+140',
  'km final': '215+120',
  'Status': 'Identificado',
  'Equipe': 'Civil: SGOAC',
  'Encontrado em': datetime.datetime(2022, 10, 22, 0, 0),
  'Executado em': datetime.datetime(2022, 10, 22, 0, 0),
  'Sentido': 'Norte',
  'Classe': 'Drenagem Profunda',
  'Faixa': 'Não se aplica',
  'Rodovia': 'BR-116 RJ',
  'UF': 'RJ',
  'Índice': 1,
  'Elemento': 'OAC',
  'Extensão': 71.89,
  'Formato': 'TUBULAR',
  'Diâmetro montante (m)': '0.60',
  'Tipo de Bueiro (Tubular)': 'BSTC',
  'Tipo de Bueiro (Celular)': None,
  'Marginal': 'Não',
  'Latitude Montante': -22.7100723,
  'Longitude Montante': -43.7363794,
  'Latitude Jusante': -22.7101837,
  'Longitude Jusante': -43.735782,
  'X UTM': 629780.12,
  'Y UTM': 7488022,
  'X UTM Jusante': 629841.38,
  'Y UTM Jusante': 7488009.14,
  'Zona': '23K',
  'Novo Cadastro': 'Não',
  'Rodovia (Montante)': 'BR-116',
  'Rodovia (Jusante)': 'BR-116',
  'Km (Montante)': '215+140',
  'Data d

Criando itens de Inventário

In [47]:
objects=[]
erros=[]

for index, a in enumerate(tqdm(values[500:])):
#     print(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']))
    km=float(a['km'].replace('+','.'))
    end_km=float(a['km final'].replace('+','.'))
    point, road = get_road_coordinates(a['Rodovia'], km, str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])), company)
    
    try:
        
        rep=Reporting(
                    company=company,
                    occurrence_type=occurrence_type_ficha,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    road_name=a['Rodovia'],
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
                    found_at=a['Encontrado em'],
                    executed_at=a['Executado em'],
                    km=float(km),
                    end_km_manually_specified=True if a['km final'] is not None else False,
                    end_km=float(end_km) if a['km final'] is not None else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        "id_ccr_antt": a["Identificação ANTT"],
                        "city": a.get("Município",''),
                        "uf": a.get("UF",''),
                        "index": a["Índice"],
                        "element": a["Elemento"],
                        "length": a["Extensão"],
                        "format": get_value('Formato', occurrence_type_ficha, a["Formato"].capitalize()),
                        "other_format": a.get("Outro Formato"),
                        "hole_kind_celular": get_value('Tipo de Bueiro (Celular)', occurrence_type_ficha, a["Tipo de Bueiro (Celular)"]) if a["Tipo de Bueiro (Celular)"] is not None else None,
                        "hole_kind_tubular": get_value('Tipo de Bueiro (Tubular)', occurrence_type_ficha, a["Tipo de Bueiro (Tubular)"]) if a["Tipo de Bueiro (Tubular)"] is not None else None,
                        "hole_kind_ovoide": get_value('Tipo de Bueiro (Ovóide)', occurrence_type_ficha, a["Tipo de Bueiro (Ovóide)"]) if a.get("Tipo de Bueiro (Ovóide)") is not None else None,
                        "hole_kind_other": get_value('Tipo de Bueiro', occurrence_type_ficha, a["Tipo de Bueiro"]) if a.get("Tipo de Bueiro") is not None else None,
                        "passarel": get_value('Travessia', occurrence_type_ficha, a["Travessia"]) if a.get("Travessia") is not None else None,
                        "marginal": get_value('Marginal', occurrence_type_ficha, a["Marginal"]) if a.get("Marginal") is not None else None,
                        "notes": a["Observações"],
                        "description_element": a.get("Descrição do Elemento"),
                        "na_faixa_dominio": True if a.get("Está na Faixa de Domínio?")=='Sim' else False                                                    
                    })
        rep.save()
        
        # Verificação Montante, caso não exista cria
        
        rep_montante=Reporting.objects.filter(occurrence_type=occurrence_type_elemento,form_data__code=a.get('Código Montante'))
        
        if not rep_montante.exists() and a.get('Código Montante'):
            if a.get('Latitude Montante') and a.get('Longitude Montante'):
                _,_,point,_ =point_to_km(float(a.get('Longitude Montante')),float(a.get('Latitude Montante')),a['Rodovia'])
            rep_montante=Reporting(
                        company=company,
                        occurrence_type=occurrence_type_elemento,
                        lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                        road=road,
                        road_name=a['Rodovia'],
                        direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                        created_by=user,
                        found_at=a['Data de Cadastro (Montante)'],
                        executed_at=a['Executado em'],
                        km=float(a['Km (Montante)'].replace('+','.')),
#                         end_km_manually_specified=True if a['km final'] is not None else False,
#                         end_km=float(a['km final']) if a['km final'] is not None else None,
                        point=point,
                        firm=firm,
                        status=status,
                        form_data={
                            "latitude": float(a["Latitude Montante"]),
                            "longitude": float(a["Longitude Montante"]),
                            "zone": get_value('Zona', occurrence_type_elemento, a["Zona"]) if a.get('Zona') else None,
                            "x_utm": float(a["X UTM"]) if a.get("X UTM") else None,
                            "y_utm": float(a["Y UTM"]) if a.get("Y UTM") else None,
                            "new_register": True if a["Novo Cadastro"]=='Sim' else False,
                            "index": a["Índice (Montante)"],
                            "code": a.get('Código Montante'),
                            "entry_struc": get_value('Estrutura Entrada', occurrence_type_elemento, a["Estrutura Entrada"]) if a.get('Estrutura Entrada') else None,
                            "ambient": get_value('Ambiente', occurrence_type_elemento, a["Ambiente Montante"]) if a.get('Ambiente Montante') else None,
                            "height": a["Altura montante (m)"],
                            "width": a["Largura montante (m)"],
                            "diameter": a["Diâmetro montante (m)"],
                            "material_rev": get_value('Material Revestimento', occurrence_type_elemento, a["Material Revestimento Montante"]) if a.get('Material Revestimento Montante') else None,
                            "description_mat": a.get("Descrição do Material"),
                            "description_element": a.get("Descrição do Elemento"),
                            "side": get_value('Lado', occurrence_type_elemento, a["Lado Montante"]) if a.get('Lado Montante') else None,
#                             "picture": a.get("Foto detalhe")ImagesArray
#                             "picture_pan": a.get("Foto panoramica")ImagesArray
#                             "exclude_element": a["Elemento Excluído"]select
#                             "exclusion_motive": a["Motivo Exclusão"]textArea
                            "notes": a["Observações"],
#                             "dimension": a["Dimensão (m)"]float                                               
                        })
            
            rep_montante.save()
        else:
            rep_montante = rep_montante.first()
            
            
        # Verificação Jusante, caso não exista cria
        
        rep_jusante=Reporting.objects.filter(occurrence_type=occurrence_type_elemento,form_data__code=a.get('Código Jusante'))
        
        if not rep_jusante.exists() and a.get('Código Jusante'):
            if a.get('Latitude Jusante') and a.get('Longitude Jusante'):
                _,_,point,_ =point_to_km(float(a.get('Longitude Jusante')),float(a.get('Latitude Jusante')),a['Rodovia'])
            rep_jusante=Reporting(
                        company=company,
                        occurrence_type=occurrence_type_elemento,
                        lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                        road=road,
                        road_name=a['Rodovia'],
                        direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                        created_by=user,
                        found_at=a['Data de Cadastro (Jusante)'],
                        executed_at=a['Executado em'],
                        km=float(a['Km (Jusante)'].replace('+','.')),
#                         end_km_manually_specified=True if a['km final'] is not None else False,
#                         end_km=float(a['km final']) if a['km final'] is not None else None,
                        point=point,
                        firm=firm,
                        status=status,
                        form_data={
                            "latitude": float(a["Latitude Jusante"]),
                            "longitude": float(a["Longitude Jusante"]),
                            "zone": get_value('Zona', occurrence_type_elemento, a["Zona"]) if a.get('Zona') else None,
                            "x_utm": float(a["X UTM"]) if a.get("X UTM") else None,
                            "y_utm": float(a["Y UTM"]) if a.get("Y UTM") else None,
                            "new_register": True if a["Novo Cadastro"]=='Sim' else False,
                            "index": a["Índice (Jusante)"],
                            "code": a.get('Código Jusante'),
                            "entry_struc": get_value('Estrutura Entrada', occurrence_type_elemento, a["Estrutura Saída"]) if a.get('Estrutura Saída') else None,
                            "ambient": get_value('Ambiente', occurrence_type_elemento, a["Ambiente Jusante"]) if a.get('Ambiente Jusante') else None,
                            "height": a["Altura Jusante (m)"],
                            "width": a["Largura Jusante (m)"],
#                             "diameter": a["Diâmetro jusante (m)"],
                            "material_rev": get_value('Material Revestimento', occurrence_type_elemento, a["Material Revestimento Montante"]) if a.get('Material Revestimento Montante') else None,
                            "description_mat": a.get("Descrição do Material"),
                            "description_element": a.get("Descrição do Elemento"),
                            "side": get_value('Lado', occurrence_type_elemento, a["Lado Jusante"]) if a.get('Lado Jusante') else None,
#                             "picture": a.get("Foto detalhe")ImagesArray
#                             "picture_pan": a.get("Foto panoramica")ImagesArray
#                             "exclude_element": a["Elemento Excluído"]select
#                             "exclusion_motive": a["Motivo Exclusão"]textArea
                            "notes": a["Observações"],
#                             "dimension": a["Dimensão (m)"]float                                               
                        })
            
            rep_jusante.save()
        else:
            rep_jusante = rep_jusante.first()
        
        
        # Relação ReportingInReporting e ReportingInRelation
        if a.get('Código Montante') and rep_montante:
            rep_relation = ReportingRelation.objects.get(name='Montante')
            rep_in_rep_montante=ReportingInReporting(parent=rep,child=rep_montante,reporting_relation=rep_relation)
            rep_in_rep_montante.save()
        if a.get('Código Jusante') and rep_jusante:
            rep_relation = ReportingRelation.objects.get(name='Jusante')
            rep_in_rep_jusante=ReportingInReporting(parent=rep,child=rep_jusante,reporting_relation=rep_relation)
            rep_in_rep_jusante.save()
            
    except Exception as e:
        print(f'Linha: {index+2} - {e}')
        erros.append(a)

  0%|          | 0/7536 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching K

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-10-17 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-10-17 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-10-17 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Searching KM 

...

Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-11-19 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-11-19 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-11-19 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Found K

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-11-02 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-11-02 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-11-02 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching K

In [48]:
import pandas as pd
pd.DataFrame(data=erros).to_excel(filename+'_erros.xlsx')

Salvando itens de Inventário no sistema e imprimindo os seriais

In [115]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/42 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28887
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28888
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28889
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28890
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28891
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28892
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28893
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28894
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CS

In [33]:
erros

[{'Identificação ANTT': 'BU 116 RJ 215+140 N 1',
  'km': '215+140',
  'km final': '215+120',
  'Status': 'Identificado',
  'Equipe': 'Civil: SGOAC',
  'Encontrado em': datetime.datetime(2022, 10, 22, 0, 0),
  'Executado em': datetime.datetime(2022, 10, 22, 0, 0),
  'Sentido': 'Norte',
  'Classe': 'Drenagem Profunda',
  'Faixa': 'Não se aplica',
  'Rodovia': 'BR-116 RJ',
  'UF': 'RJ',
  'Índice': 1,
  'Elemento': 'OAC',
  'Extensão': 71.89,
  'Formato': 'TUBULAR',
  'Diâmetro montante (m)': '0.60',
  'Tipo de Bueiro (Tubular)': 'BSTC',
  'Tipo de Bueiro (Celular)': None,
  'Marginal': 'Não',
  'Latitude Montante': -22.7100723,
  'Longitude Montante': -43.7363794,
  'Latitude Jusante': -22.7101837,
  'Longitude Jusante': -43.735782,
  'X UTM': 629780.12,
  'Y UTM': 7488022,
  'X UTM Jusante': 629841.38,
  'Y UTM Jusante': 7488009.14,
  'Zona': '23K',
  'Novo Cadastro': 'Não',
  'Rodovia (Montante)': 'BR-116',
  'Rodovia (Jusante)': 'BR-116',
  'Km (Montante)': '215+140',
  'Data de Cadas